# Replicate | レプリケート

> [Replicate](https://replicate.com/blog/machine-learning-needs-better-tools)は、クラウド上で機械学習モデルを実行するサービスです。わずか数行のコードで実行可能なオープンソースモデルのライブラリを提供しています。自分の機械学習モデルを構築している場合、Replicateはそれをスケールアップして展開することを容易にします。
>
> > [Replicate](https://replicate.com/blog/machine-learning-needs-better-tools) runs machine learning models in the cloud. We have a library of open-source models that you can run with a few lines of code. If you're building your own machine learning models, Replicate makes it easy to deploy them at scale.

この例では、LangChainを使用して`Replicate` [モデル](https://replicate.com/explore)と対話する方法について説明します。

> This example goes over how to use LangChain to interact with `Replicate` [models](https://replicate.com/explore)



## Setup | セットアップ



In [1]:
# magics to auto-reload external modules in case you are making changes to langchain while working on this notebook
%load_ext autoreload
%autoreload 2

このノートブックを実行するには、[replicate](https://replicate.com)のアカウントを作成し、[replicate python client](https://github.com/replicate/replicate-python)をインストールする必要があります。

> To run this notebook, you'll need to create a [replicate](https://replicate.com) account and install the [replicate python client](https://github.com/replicate/replicate-python).



In [7]:
!poetry run pip install replicate

  Using cached replicate-0.9.0-py3-none-any.whl (21 kB)


In [3]:
# get a token: https://replicate.com/account

from getpass import getpass

REPLICATE_API_TOKEN = getpass()

In [4]:
import os

os.environ["REPLICATE_API_TOKEN"] = REPLICATE_API_TOKEN

In [18]:
from langchain.chains import LLMChain
from langchain.llms import Replicate
from langchain.prompts import PromptTemplate

## Calling a model | モデルを呼び出す

[replicate explore page](https://replicate.com/explore)でモデルを探し、次の形式でモデル名とバージョンを貼り付けてください: model\_name/version。

> Find a model on the [replicate explore page](https://replicate.com/explore), and then paste in the model name and version in this format: model\_name/version.

例えば、こちらが[`LLama-V2`](https://replicate.com/a16z-infra/llama13b-v2-chat)です。

> For example, here is [`LLama-V2`](https://replicate.com/a16z-infra/llama13b-v2-chat).



In [19]:
llm = Replicate(
    model="a16z-infra/llama13b-v2-chat:df7690f1994d94e96ad9d568eac121aecf50684a0b0963b25a41cc40061269e5",
    model_kwargs={"temperature": 0.75, "max_length": 500, "top_p": 1},
)
prompt = """
User: Answer the following yes/no question by reasoning step by step. Can a dog drive a car?
Assistant:
"""
llm(prompt)

'1. Dogs do not have the ability to operate complex machinery like cars.\n2. Dogs do not have human-like intelligence or cognitive abilities to understand the concept of driving.\n3. Dogs do not have the physical ability to use their paws to press pedals or turn a steering wheel.\n4. Therefore, a dog cannot drive a car.'

別の例として、この[dolly model](https://replicate.com/replicate/dolly-v2-12b)については、APIタブをクリックしてください。モデル名/バージョンは次のとおりです：`replicate/dolly-v2-12b:ef0e1aefc61f8e096ebe4db6b2bacc297daf2ef6899f0f7e001ec445893500e5`

> As another example, for this [dolly model](https://replicate.com/replicate/dolly-v2-12b), click on the API tab. The model name/version would be: `replicate/dolly-v2-12b:ef0e1aefc61f8e096ebe4db6b2bacc297daf2ef6899f0f7e001ec445893500e5`

`model` パラメータのみが必要ですが、初期化する際に他のモデルパラメータを追加することもできます。

> Only the `model` param is required, but we can add other model params when initializing.

例えば、私たちが安定化拡散を実行していて、画像のサイズを変更したいとしたら：

> For example, if we were running stable diffusion and wanted to change the image dimensions:

```
Replicate(model="stability-ai/stable-diffusion:db21e45d3f7023abc2a46ee38a23973f6dce16bb082a930b0c49861f96d1e5bf", input={'image_dimensions': '512x512'})
```

モデルの最初の出力のみが返されることに注意してください。

> *Note that only the first output of a model will be returned.*



In [20]:
llm = Replicate(
    model="replicate/dolly-v2-12b:ef0e1aefc61f8e096ebe4db6b2bacc297daf2ef6899f0f7e001ec445893500e5"
)

In [21]:
prompt = """
Answer the following yes/no question by reasoning step by step. 
Can a dog drive a car?
"""
llm(prompt)

'No, dogs lack some of the brain functions required to operate a motor vehicle. They cannot focus and react in time to accelerate or brake correctly. Additionally, they do not have enough muscle control to properly operate a steering wheel.\n\n'

この構文を使用して、任意のレプリケートモデルを呼び出すことができます。例えば、ステーブルディフュージョンを呼び出すことが可能です。

> We can call any replicate model using this syntax. For example, we can call stable diffusion.



In [22]:
text2image = Replicate(
    model="stability-ai/stable-diffusion:db21e45d3f7023abc2a46ee38a23973f6dce16bb082a930b0c49861f96d1e5bf",
    model_kwargs={"image_dimensions": "512x512"},
)

In [23]:
image_output = text2image("A cat riding a motorcycle by Picasso")
image_output

'https://pbxt.replicate.delivery/bqQq4KtzwrrYL9Bub9e7NvMTDeEMm5E9VZueTXkLE7kWumIjA/out-0.png'

モデルがURLを出力します。それをレンダリングしましょう。

> The model spits out a URL. Let's render it.



In [24]:
!poetry run pip install Pillow


[notice] A new release of pip is available: 23.2 -> 23.2.1
[notice] To update, run: pip install --upgrade pip


In [ ]:
from io import BytesIO

import requests
from PIL import Image

response = requests.get(image_output)
img = Image.open(BytesIO(response.content))

img

## Streaming Response | ストリーミングレスポンス

必要に応じて、生成されるレスポンスをストリーミングできます。これは、時間を要する生成作業においてユーザーにインタラクティビティを示すのに役立ちます。詳細は[Streaming](https://python.langchain.com/docs/modules/model_io/llms/how_to/streaming_llm)のドキュメントをご覧ください。

> You can optionally stream the response as it is produced, which is helpful to show interactivity to users for time-consuming generations. See detailed docs on [Streaming](https://python.langchain.com/docs/modules/model_io/llms/how_to/streaming_llm) for more information.



In [26]:
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

llm = Replicate(
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
    model="a16z-infra/llama13b-v2-chat:df7690f1994d94e96ad9d568eac121aecf50684a0b0963b25a41cc40061269e5",
    model_kwargs={"temperature": 0.75, "max_length": 500, "top_p": 1},
)
prompt = """
User: Answer the following yes/no question by reasoning step by step. Can a dog drive a car?
Assistant:
"""
_ = llm(prompt)

1. Dogs do not have the physical ability to operate a vehicle.

# Stop Sequences | ストップシーケンス

また、停止シーケンスを指定することもできます。もし、解析する予定の生成物に明確な停止シーケンスがある場合、指定された `max_length` までモデルが余計なことを言い続けるのを待つよりも、一つまたは複数の停止シーケンスに到達した時点で生成を終了する方が（コストも速度も）より良いです。停止シーケンスは、ストリーミングモードであろうとなかろうと関係なく機能し、Replicateは停止シーケンスまでの生成にのみ課金します。

> You can also specify stop sequences. If you have a definite stop sequence for the generation that you are going to parse with anyway, it is better (cheaper and faster!) to just cancel the generation once one or more stop sequences are reached, rather than letting the model ramble on till the specified `max_length`. Stop sequences work regardless of whether you are in streaming mode or not, and Replicate only charges you for the generation up until the stop sequence.



In [27]:
import time

llm = Replicate(
    model="a16z-infra/llama13b-v2-chat:df7690f1994d94e96ad9d568eac121aecf50684a0b0963b25a41cc40061269e5",
    model_kwargs={"temperature": 0.01, "max_length": 500, "top_p": 1},
)

prompt = """
User: What is the best way to learn python?
Assistant:
"""
start_time = time.perf_counter()
raw_output = llm(prompt)  # raw output, no stop
end_time = time.perf_counter()
print(f"Raw output:\n {raw_output}")
print(f"Raw output runtime: {end_time - start_time} seconds")

start_time = time.perf_counter()
stopped_output = llm(prompt, stop=["\n\n"])  # stop on double newlines
end_time = time.perf_counter()
print(f"Stopped output:\n {stopped_output}")
print(f"Stopped output runtime: {end_time - start_time} seconds")

Raw output:
 There are several ways to learn Python, and the best method for you will depend on your learning style and goals. Here are a few suggestions:

1. Online tutorials and courses: Websites such as Codecademy, Coursera, and edX offer interactive coding lessons and courses that can help you get started with Python. These courses are often designed for beginners and cover the basics of Python programming.
2. Books: There are many books available that can teach you Python, ranging from introductory texts to more advanced manuals. Some popular options include "Python Crash Course" by Eric Matthes, "Automate the Boring Stuff with Python" by Al Sweigart, and "Python for Data Analysis" by Wes McKinney.
3. Videos: YouTube and other video platforms have a wealth of tutorials and lectures on Python programming. Many of these videos are created by experienced programmers and can provide detailed explanations and examples of Python concepts.
4. Practice: One of the best ways to learn Pytho

## Chaining Calls | メソッドチェーン

langchainの全体的な趣旨は...チェーンを作ることです！その方法を以下の例で示します。

> The whole point of langchain is to... chain! Here's an example of how do that.



In [28]:
from langchain.chains import SimpleSequentialChain

まず、このモデルのLLMをflan-5と定義し、text2imageをstable diffusionモデルとしましょう。

> First, let's define the LLM for this model as a flan-5, and text2image as a stable diffusion model.



In [29]:
dolly_llm = Replicate(
    model="replicate/dolly-v2-12b:ef0e1aefc61f8e096ebe4db6b2bacc297daf2ef6899f0f7e001ec445893500e5"
)
text2image = Replicate(
    model="stability-ai/stable-diffusion:db21e45d3f7023abc2a46ee38a23973f6dce16bb082a930b0c49861f96d1e5bf"
)

チェーン内の最初のプロンプト

> First prompt in the chain



In [30]:
prompt = PromptTemplate(
    input_variables=["product"],
    template="What is a good name for a company that makes {product}?",
)

chain = LLMChain(llm=dolly_llm, prompt=prompt)

会社の説明のためのロゴを取得するための二番目のプロンプト

> Second prompt to get the logo for company description



In [31]:
second_prompt = PromptTemplate(
    input_variables=["company_name"],
    template="Write a description of a logo for this company: {company_name}",
)
chain_two = LLMChain(llm=dolly_llm, prompt=second_prompt)

3番目のプロンプトでは、プロンプト2から出力された説明に基づいて画像を作成しましょう

> Third prompt, let's create the image based on the description output from prompt 2



In [32]:
third_prompt = PromptTemplate(
    input_variables=["company_logo_description"],
    template="{company_logo_description}",
)
chain_three = LLMChain(llm=text2image, prompt=third_prompt)

それでは、実行しましょう！

> Now let's run it!



In [33]:
# Run the chain specifying only the input variable for the first chain.
overall_chain = SimpleSequentialChain(
    chains=[chain, chain_two, chain_three], verbose=True
)
catchphrase = overall_chain.run("colorful socks")
print(catchphrase)



> Entering new SimpleSequentialChain chain...
Colorful socks could be named after a song by The Beatles or a color (yellow, blue, pink). A good combination of letters and digits would be 6399. Apple also owns the domain 6399.com so this could be reserved for the Company.


A colorful sock with the numbers 3, 9, and 99 screen printed in yellow, blue, and pink, respectively.


https://pbxt.replicate.delivery/P8Oy3pZ7DyaAC1nbJTxNw95D1A3gCPfi2arqlPGlfG9WYTkRA/out-0.png

> Finished chain.
https://pbxt.replicate.delivery/P8Oy3pZ7DyaAC1nbJTxNw95D1A3gCPfi2arqlPGlfG9WYTkRA/out-0.png


In [ ]:
response = requests.get(
    "https://replicate.delivery/pbxt/682XgeUlFela7kmZgPOf39dDdGDDkwjsCIJ0aQ0AO5bTbbkiA/out-0.png"
)
img = Image.open(BytesIO(response.content))
img